<a id=0></a>
# 7.Categorical Features
カテゴリカル特徴量（変数）の取り扱い

---
### [1.LabelEncoder()](#1)
### [2.get_dummies()](#2)
### [3.OneHotEncoder()](#3)
### [4.pd.get_dummies()とOneHotEncoder()の違い](#4)
### [5.Seriesのstr属性を使う](#5)

---

データセットとしてsample1_without_index.csvを使用する

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# google colaboratoryの場合はドライブをマウントして、適切なパスを指定してください
df = pd.read_csv('./sample1_without_index.csv')
df.head()

In [ ]:
df = df[['Color', 'Shape']]

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df[df['Color'].isnull()].index

---
<a id=1></a>
[Topへ](#0)

---
## 1. LabelEncoder()  
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html  
※ ラベルを数値(0, 1, 2, ...)で置換する

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# インスタンスもしくはオブジェクトと呼ばれる
encoder = LabelEncoder()

In [ ]:
# フィット、適合・学習させる
encoder.fit(df['Color'])

In [ ]:
# フィットさせた結果、Colorの持つクラスを記憶する
encoder.classes_

In [ ]:
# transformで変換を行う
encoder.transform(df['Color'])

In [ ]:
df.head()

In [ ]:
# 各色とNaNを含むレコードを確認する
df_ce = df.copy()
df_ce['Color_encoded'] = encoder.transform(df['Color'])
df_ce = df_ce[['Color', 'Color_encoded', 'Shape']]
df_ce.loc[36:42]

In [ ]:
# 逆変換
encoder.inverse_transform(df_ce.loc[36:42, 'Color_encoded'])

---
<a id=2></a>
[Topへ](#0)

---
## 2. get_dummies()  
https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html  
※　カテゴリー変数をダミー変数化（0 or 1）する

* ダミー変数化を実行
* drop_first=Trueとは
* np.nanはどうなるのか
---

ダミー変数化を実行

In [ ]:
pd.get_dummies(df['Color']).head()

drop_first=Trueとは  

In [ ]:
# blueのカラムがない場合でも0, 0ならばblueと判別できる
pd.get_dummies(df['Color'], drop_first=True).head()

In [ ]:
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True)

In [ ]:
df_cd.head()

In [ ]:
# 複数カラムを同時に変換、さらにデータフレーム形式で返ってくるため便利
pd.get_dummies(df, columns=['Color', 'Shape'], drop_first=True)

np.nanはどうなるのか

In [ ]:
df_cd.isnull().sum()

In [ ]:
# nanとblueの区別がつかない
df_cd.loc[36:42]

In [ ]:
# dummy_naを使用することで解消
df_cd = pd.get_dummies(df, columns=['Color'], drop_first=True, dummy_na=True)
df_cd.loc[36:42]

# nanを含まないように前処理を行うのがよいが、どうしてもnanを含めざる得ない場合はこれを使用する

---
<a id=3></a>
[Topへ](#0)

---
## 3. OneHotEncoder()  
※　One-hot : ひとつが1で他は0  
※　pd.get_dummies()にはない機能を使ってダミー変数化を行う

https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html

デフォルトのKeyword Argument : drop=None, handle_unknown='error'

* OneHotEncoder()を使ってみる
* 複数の特徴量を変換
---

OneHotEncoder()を使ってみる

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()

In [ ]:
encoder.fit(df[['Color']])

In [ ]:
encoder.categories_

In [ ]:
encoder.transform(df[['Color']])
# Sparse : まばらな、疎な
# メモリの消費が減るため、sparse=Trueが初期設定となっている

In [ ]:
# アレイの確認方法
encoder.transform(df[['Color']]).toarray()[:5]

複数の特徴量を変換

In [ ]:
encoder = OneHotEncoder()

In [ ]:
encoder.fit(df)

In [ ]:
encoder.categories_

In [ ]:
encoder.transform(df).toarray()[:5]
# DataFrame形式でないのは、ndarrayをモデルにfitさせるのが効率的（無駄がない）身体と思われる。人にとってはDataFrameのほうが見やすくはあるが

In [ ]:
encoder.inverse_transform([[0, 1, 0, 0, 0, 1, 0, 0]])

---
<a id=4></a>
[Topへ](#0)

---
## 4. pd.get_dummies()とOneHotEncoder()の違い

* get_dummies()ではトレインセットとテストセットに差が生じる
* OneHotEncoder(handle_unknown='error', drop='first')の場合
* OneHotEncoder(handle_unknown='ignore')の場合
---

get_dummies()ではトレインセットとテストセットに差が生じる

In [ ]:
np.random.seed(1)
s = pd.Series(np.random.choice([0, 1], len(df)), name='target')
s

In [ ]:
df_new = pd.concat([df, s], axis=1)
df_new.head()

In [ ]:
# トレインテストスプリット
from sklearn.model_selection import train_test_split

In [ ]:
# y : 目的変数、X : 説明変数
y = df_new.pop('target')   # pop()では元のdf_newからtargetが切り出される
X = df_new

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, stratify=y, random_state=17)
# stratify : 階層化。yの分離に偏りをなくす。train, testで0, 1の割合を均等にする

In [ ]:
X_test
# testにはtrainにあるgreen, squareが存在しない
# この場合はどうなるのか？

In [ ]:
pd.get_dummies(X_train, drop_first=True, dummy_na=True).head()

In [ ]:
pd.get_dummies(X_test, drop_first=True, dummy_na=True).head()
# 特徴量の数、項目が変化してしまう。テストデータはモデルに当てはめるデータとして使用できない。

In [ ]:
encoder = OneHotEncoder(drop='first')

In [ ]:
encoder.fit_transform(X_train).toarray()[:5]

In [ ]:
encoder.transform(X_test).toarray()
# encoderが記憶しているため、カラム数と項目は一致する

OneHotEncoder(handle_unknown='error', drop='first')の場合

In [ ]:
# ‘error’ : Raise an error if an unknown category is present during transform.
# ‘ignore’ : When an unknown category is encountered during transform, the resulting one-hot encoded columns for this feature will be all zeros. In the inverse transform, an unknown category will be denoted as None.

In [ ]:
encoder_error = OneHotEncoder(handle_unknown='error', drop='first')

In [ ]:
encoder_error.fit_transform(X_train).toarray()[:5]

In [ ]:
encoder_error.transform(X_test).toarray()

In [ ]:
# trainにはない値がtestに存在するという状況を作る
X_test_new = X_test.copy()
X_test_new.loc[16, 'Color'] = 'purple'
X_test_new

In [ ]:
# encoder_error.transform(X_test_new)
# エラーを発生させることで対応する

OneHotEncoder(handle_unknown='ignore')の場合

In [ ]:
encoder_ignore = OneHotEncoder(handle_unknown='ignore')
# drop='first'とは共存できない

In [ ]:
encoder_ignore.fit(X_train)

In [ ]:
encoder_ignore.transform(X_test).toarray()

In [ ]:
encoder_ignore.transform(X_test_new).toarray()
# 未知の値に対してはすべてを0とすることで区別をする

#### 状況に応じて使い分ける（例）
* 分類される値が少ない、レコード量が多い  
    ＝＞　testデータに欠ける値はない　＝＞　get_dummies, OneHotEncoder(drop='first')
* 分類される値が少ない、レコード量が少ない  
    ＝＞　testデータに欠ける値があるかもしれない　＝＞　OneHotEncoder(handle_unknown='error', drop='first')
* 分類される値が多い、レコード量が少ない  
    ＝＞　testデータにtrainデータにない値が確実に入る　＝＞ OneHotEncoder, handle_unknown='ignore'

---
<a id=5></a>
[Topへ](#0)

---
## 5.Seriesのstr属性を使う

* Series.strとは
* メソッドを確認
* 利用頻度の高い置換、抽出、分離
---

Series.strとは

In [ ]:
df = pd.DataFrame()
df['ID'] = ['A-123', 'B-456', 'A-789', 'B-123']
df['Color'] = ['py/white black', 'red green blue', 'py/yellow', 'purple white']
df

In [ ]:
df['ID'].str
# df.str

In [ ]:
df['ID'].str[:3]

メソッドを確認

In [ ]:
df['ID'].str.lower()

In [ ]:
df['ID'].str.startswith('B')
# endswith

In [ ]:
df['Color'].str.contains('white')

In [ ]:
# もしくは
df['Color'].str.contains('ye|pu')

利用頻度の高い置換、抽出、分離

In [ ]:
df['Color'].str.replace('black', 'gold')

In [ ]:
df['ID'].str.split('-')

In [ ]:
# expandという引数がある
df['ID'].str.split('-', expand=True)

In [ ]:
df[['ID_a', 'ID_n']] = df['ID'].str.split('-', expand=True)
df

In [ ]:
df[['Color_1', 'Color_2', 'Color_3']] = df['Color'].str.split(' ', expand=True)
df

In [ ]:
# 正規表現を使うことも可能
df['Color_1'].str.extract('(py/)', expand=True)

In [ ]:
df['py'] = df['Color_1'].str.extract('(py/)', expand=True)
df

In [ ]:
df['Color_1'] = df['Color_1'].str.replace('py/', '')
df

---
[Topへ](#0)

---
## 以上
    
---